In [74]:
# Import all connection libraries
import spotipy
from spotipy import SpotifyClientCredentials
import sqlalchemy as sql
from sqlalchemy import table, column
import psycopg2

# Import all data manipulation libraries

import dask.dataframe as dd
import pandas as pd
from pandarallel import pandarallel
from tqdm import tqdm
from datetime import datetime
pandarallel.initialize(nb_workers=8, verbose=0)

In [75]:
# establish connection to Spotify API

cid = 'e5448a8a4fdc4b5d98b44e956d50546d'
secret = '8924c0394d3f49a4a569fc03e891aa1b'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

In [76]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

In [77]:
# load training data
new_batch = pd.read_csv('distinct_playlists_new.csv')[['playlist_uri', 'playlist_name']]
new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = new_batch.merge(db_query, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

new_batch = pd.DataFrame(anti_join)
new_batch = new_batch[5:10]
new_batch.head(10)

,playlist_uri,playlist_name
1776,spotify:playlist:1SVkWpqPP87mWELCn8pM5q,Pokemon EDM
1777,spotify:playlist:1SWClwMGIJVORmvqjg2iBg,STAR WALKIN CLEAN
1778,spotify:playlist:1SWvsuHyBubD9Y0yosPqHD,HUGEL - Morenita | House with Groove????
1779,spotify:playlist:1SyiKOqGozEr9vy6aPWIPP,EDM - Headbanger Hits
1780,spotify:playlist:1SZ1Y4t53nydliBjkPs1ng,Trucker Country Songs


In [78]:
import dask
data = sp.playlist_tracks('spotify:playlist:1SyiKOqGozEr9vy6aPWIPP')['items']
dfs = [dask.delayed(pd.io.json.json_normalize)(x) for x in data]
df = dd.from_delayed(dfs)
df.head()

c:\Users\Chase\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\core.py:119: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  return func(*(_execute_task(a, cache) for a in args))
c:\Users\Chase\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\core.py:119: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  return func(*(_execute_task(a, cache) for a in args))
c:\Users\Chase\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\core.py:7593: UserWarning: Insufficient elements for `head`. 5 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,...,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2018-02-23T22:06:42Z,False,None,https://open.spotify.com/user/12183971869,https://api.spotify.com/v1/users/12183971869,12183971869,user,spotify:user:12183971869,album,[{'external_urls': {'spotify': 'https://open.s...,...,4mcCRt9T9mW5MFldNdyF1I,False,Final Boss,0,None,True,14,track,spotify:track:4mcCRt9T9mW5MFldNdyF1I,None


In [136]:

playlist = 'spotify:playlist:1SyiKOqGozEr9vy6aPWIPP'
tracks = get_playlist_tracks('spotify:playlist:1SyiKOqGozEr9vy6aPWIPP')

if len(tracks) >= 100:
        data2 = sp.playlist_tracks(playlist, offset=100)['items']
        dfs2 = [dask.delayed(pd.json_normalize)(x) for x in data2]
        tracks2 = dd.from_delayed(dfs2)
        data3 = sp.playlist_tracks(playlist, offset=200)['items']
        dfs3 = [dask.delayed(pd.json_normalize)(x) for x in data3]
        tracks3 = dd.from_delayed(dfs3)
        tracks = dd.multi.concat([tracks, tracks2, tracks3])
        secondary_data = sp.playlist_tracks(playlist, offset=100)['items']
        dfs4 = [dask.delayed(pd.json_normalize)(x) for x in secondary_data]
        secondary_data = dd.from_delayed(dfs4)
        secondary_data['uri'] = playlist
        tracks['playlist_uri'] = playlist
        tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
        tracks = tracks.rename(columns={"name_x":"playlist_name", "track.name_x":"track_name", "track.uri_x":"track_uri", "track.album.name_x":"album_name", "track.explicit_x":"isExplicit", "track.album.release_date_x":"release_date", "track.duration_ms_x":"duration_ms", "track.album.uri_x":"album_uri", "added_by.external_urls.spotify_x": "added_by_external_urls_spotify", 'added_by.href_x':"added_by_href", "added_by.id_x":"added_by_id", "added_by.type_x":"added_by_type", "added_by.uri_x":"added_by_uri", "track.album.album_type_x":"track_album_album_type", "track.album.external_urls.spotify_x":"track_album_external_urls_spotify", "track.album.href_x":"track_album_href", "track.album.id_x":"track_album_id", "track.album.release_date_precision_x":"track_album_release_date_precision", "track.album.total_tracks_x":"track_album_total_tracks","track.album.type_x":"track_album_type", "track.disc_number_x":"track_disc_number", "track.episode_x":"track_episode", "track.external_ids.isrc_x":"track_external_ids_isrc", "track.external_urls.spotify_x":"track_external_ids_spotify", "track.href_x":"track_href", "track.id_x":"track_id", "track.is_local_x":"track_is_local", "track.popularity_x":"track_popularity", "track.preview_url_x":"track_preview_url", "track.track_x":"track_track", "track.track_number_x":"track_track_number", "track.type_x":"track_type", "video_thumbnail.url_x":"video_thumbnail_url", "external_urls.spotify_x":"external_urls_spotify", "followers.href_x":"followers_href", "followers.total_x":"followers_total", "owner.display_name_x":"owner_display_name", "owner.external_urls.spotify_x":"owner_external_urls_spotify", "owner.href_x":"owner_href", "owner.id_x":"owner_id", "owner.type_x":"owner_type", "owner.uri_x":"owner_uri", "tracks.href_x":"tracks_href", "tracks.limit_x":"tracks_limit", "tracks.next_x":"tracks_next", "tracks.offset_x":"tracks_offset", "tracks.previous_x":"tracks_previous", "tracks.total_x":"tracks_total","name_y":"playlist_name", "track.name_y":"track_name", "track.uri_y":"track_uri", "track.album.name_y":"album_name", "track.explicit_y":"isExplicit", "track.album.release_date_y":"release_date", "track.duration_ms_y":"duration_ms", "track.album.uri_y":"album_uri", "added_by.external_urls.spotify_y": "added_by_external_urls_spotify", 'added_by.href_y':"added_by_href", "added_by.id_y":"added_by_id", "added_by.type_y":"added_by_type", "added_by.uri_y":"added_by_uri", "track.album.album_type_y":"track_album_album_type", "track.album.external_urls.spotify_y":"track_album_external_urls_spotify", "track.album.href_y":"track_album_href", "track.album.id_y":"track_album_id", "track.album.release_date_precision_y":"track_album_release_date_precision", "track.album.total_tracks_y":"track_album_total_tracks","track.album.type_y":"track_album_type", "track.disc_number_y":"track_disc_number", "track.episode_y":"track_episode", "track.external_ids.isrc_y":"track_external_ids_isrc", "track.external_urls.spotify_y":"track_external_ids_spotify", "track.href_y":"track_href", "track.id_y":"track_id", "track.is_local_y":"track_is_local", "track.popularity_y":"track_popularity", "track.preview_url_y":"track_preview_url", "track.track_y":"track_track", "track.track_number_y":"track_track_number", "track.type_y":"track_type", "video_thumbnail.url_y":"video_thumbnail_url", "external_urls.spotify_y":"external_urls_spotify", "followers.href_y":"followers_href", "followers.total_y":"followers_total", "owner.display_name_y":"owner_display_name", "owner.external_urls.spotify_y":"owner_external_urls_spotify", "owner.href_y":"owner_href", "owner.id_y":"owner_id", "owner.type_y":"owner_type", "owner.uri_y":"owner_uri", "tracks.href_y":"tracks_href", "tracks.limit_y":"tracks_limit", "tracks.next_y":"tracks_next", "tracks.offset_y":"tracks_offset", "tracks.previous_y":"tracks_previous", "tracks.total_y":"tracks_total", "added_at_x":"added_at", "added_at_y":"added_at", "is_local_x":"is_local", "is_local_y":"is_local"})
        tracks = tracks.drop(columns=['track.album.artists_x', 'track.album.artists_y', 'track.album.available_markets_x', 'track.album.images_x', 'track.artists_x', 'track.available_markets_x', 'track.available_markets_y', 'track.artists_y', 'track.album.images_y','track.album.available_markets_y', 'tracl.album.images_y', 'images_x', 'tracks.items_x'], errors='ignore')

Index(['added_at', 'is_local', 'primary_color_x',
       'added_by_external_urls_spotify', 'added_by_href', 'added_by_id',
       'added_by_type', 'added_by_uri', 'track_album_album_type',
       'track_album_external_urls_spotify', 'track_album_href',
       'track_album_id', 'album_name', 'release_date',
       'track_album_release_date_precision', 'track_album_total_tracks',
       'track_album_type', 'album_uri', 'track_disc_number', 'duration_ms',
       'track_episode', 'isExplicit', 'track_external_ids_isrc',
       'track_external_ids_spotify', 'track_href', 'track_id',
       'track_is_local', 'track_name', 'track_popularity', 'track_preview_url',
       'track_track', 'track_track_number', 'track_type', 'track_uri',
       'video_thumbnail_url', 'playlist_uri', 'added_at', 'is_local',
       'primary_color_y', 'added_by_external_urls_spotify', 'added_by_href',
       'added_by_id', 'added_by_type', 'added_by_uri',
       'track_album_album_type', 'track_album_external_urls_sp

In [140]:
load_batch = []
playlist = 'spotify:playlist:1SyiKOqGozEr9vy6aPWIPP'
tracks = get_playlist_tracks('spotify:playlist:1SyiKOqGozEr9vy6aPWIPP')

if len(tracks) >= 100:
        data2 = sp.playlist_tracks(playlist, offset=100)['items']
        dfs2 = [dask.delayed(pd.json_normalize)(x) for x in data2]
        tracks2 = dd.from_delayed(dfs2)
        data3 = sp.playlist_tracks(playlist, offset=200)['items']
        dfs3 = [dask.delayed(pd.json_normalize)(x) for x in data3]
        tracks3 = dd.from_delayed(dfs3)
        tracks = dd.multi.concat([tracks, tracks2, tracks3])
        secondary_data = sp.playlist_tracks(playlist, offset=100)['items']
        dfs4 = [dask.delayed(pd.json_normalize)(x) for x in secondary_data]
        secondary_data = dd.from_delayed(dfs4)
        secondary_data['uri'] = playlist
        tracks['playlist_uri'] = playlist
        tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
        tracks = tracks.rename(columns={"name_x":"playlist_name", "track.name_x":"track_name", "track.uri_x":"track_uri", "track.album.name_x":"album_name", "track.explicit_x":"isExplicit", "track.album.release_date_x":"release_date", "track.duration_ms_x":"duration_ms", "track.album.uri_x":"album_uri", "added_by.external_urls.spotify_x": "added_by_external_urls_spotify", 'added_by.href_x':"added_by_href", "added_by.id_x":"added_by_id", "added_by.type_x":"added_by_type", "added_by.uri_x":"added_by_uri", "track.album.album_type_x":"track_album_album_type", "track.album.external_urls.spotify_x":"track_album_external_urls_spotify", "track.album.href_x":"track_album_href", "track.album.id_x":"track_album_id", "track.album.release_date_precision_x":"track_album_release_date_precision", "track.album.total_tracks_x":"track_album_total_tracks","track.album.type_x":"track_album_type", "track.disc_number_x":"track_disc_number", "track.episode_x":"track_episode", "track.external_ids.isrc_x":"track_external_ids_isrc", "track.external_urls.spotify_x":"track_external_ids_spotify", "track.href_x":"track_href", "track.id_x":"track_id", "track.is_local_x":"track_is_local", "track.popularity_x":"track_popularity", "track.preview_url_x":"track_preview_url", "track.track_x":"track_track", "track.track_number_x":"track_track_number", "track.type_x":"track_type", "video_thumbnail.url_x":"video_thumbnail_url", "external_urls.spotify_x":"external_urls_spotify", "followers.href_x":"followers_href", "followers.total_x":"followers_total", "owner.display_name_x":"owner_display_name", "owner.external_urls.spotify_x":"owner_external_urls_spotify", "owner.href_x":"owner_href", "owner.id_x":"owner_id", "owner.type_x":"owner_type", "owner.uri_x":"owner_uri", "tracks.href_x":"tracks_href", "tracks.limit_x":"tracks_limit", "tracks.next_x":"tracks_next", "tracks.offset_x":"tracks_offset", "tracks.previous_x":"tracks_previous", "tracks.total_x":"tracks_total","name_y":"playlist_name", "track.name_y":"track_name", "track.uri_y":"track_uri", "track.album.name_y":"album_name", "track.explicit_y":"isExplicit", "track.album.release_date_y":"release_date", "track.duration_ms_y":"duration_ms", "track.album.uri_y":"album_uri", "added_by.external_urls.spotify_y": "added_by_external_urls_spotify", 'added_by.href_y':"added_by_href", "added_by.id_y":"added_by_id", "added_by.type_y":"added_by_type", "added_by.uri_y":"added_by_uri", "track.album.album_type_y":"track_album_album_type", "track.album.external_urls.spotify_y":"track_album_external_urls_spotify", "track.album.href_y":"track_album_href", "track.album.id_y":"track_album_id", "track.album.release_date_precision_y":"track_album_release_date_precision", "track.album.total_tracks_y":"track_album_total_tracks","track.album.type_y":"track_album_type", "track.disc_number_y":"track_disc_number", "track.episode_y":"track_episode", "track.external_ids.isrc_y":"track_external_ids_isrc", "track.external_urls.spotify_y":"track_external_ids_spotify", "track.href_y":"track_href", "track.id_y":"track_id", "track.is_local_y":"track_is_local", "track.popularity_y":"track_popularity", "track.preview_url_y":"track_preview_url", "track.track_y":"track_track", "track.track_number_y":"track_track_number", "track.type_y":"track_type", "video_thumbnail.url_y":"video_thumbnail_url", "external_urls.spotify_y":"external_urls_spotify", "followers.href_y":"followers_href", "followers.total_y":"followers_total", "owner.display_name_y":"owner_display_name", "owner.external_urls.spotify_y":"owner_external_urls_spotify", "owner.href_y":"owner_href", "owner.id_y":"owner_id", "owner.type_y":"owner_type", "owner.uri_y":"owner_uri", "tracks.href_y":"tracks_href", "tracks.limit_y":"tracks_limit", "tracks.next_y":"tracks_next", "tracks.offset_y":"tracks_offset", "tracks.previous_y":"tracks_previous", "tracks.total_y":"tracks_total", "added_at_x":"added_at", "added_at_y":"added_at", "is_local_x":"is_local", "is_local_y":"is_local"})
        tracks = tracks.drop(columns=['track.album.artists_x', 'track.album.artists_y', 'track.album.available_markets_x', 'track.album.images_x', 'track.artists_x', 'track.available_markets_x', 'track.available_markets_y', 'track.artists_y', 'track.album.images_y','track.album.available_markets_y', 'track.album.images_y', 'images_x', 'tracks.items_x'], errors='ignore')
        
        load_batch.append(tracks)
else:
        secondary_data = sp.playlist(playlist)
        dfs4 = [dask.delayed(pd.json_normalize(x)) for x in secondary_data]
        secondary_data = dd.from_delayed(dfs4)
        secondary_data['uri'] = playlist
        tracks['playlist_uri'] = playlist
        tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri', how='inner')
        tracks = tracks.rename(columns={"name_x":"playlist_name", "track.name_x":"track_name", "track.uri_x":"track_uri", "track.album.name_x":"album_name", "track.explicit_x":"isExplicit", "track.album.release_date_x":"release_date", "track.duration_ms_x":"duration_ms", "track.album.uri_x":"album_uri", "added_by.external_urls.spotify_x": "added_by_external_urls_spotify", 'added_by.href_x':"added_by_href", "added_by.id_x":"added_by_id", "added_by.type_x":"added_by_type", "added_by.uri_x":"added_by_uri", "track.album.album_type_x":"track_album_album_type", "track.album.external_urls.spotify_x":"track_album_external_urls_spotify", "track.album.href_x":"track_album_href", "track.album.id_x":"track_album_id", "track.album.release_date_precision_x":"track_album_release_date_precision", "track.album.total_tracks_x":"track_album_total_tracks","track.album.type_x":"track_album_type", "track.disc_number_x":"track_disc_number", "track.episode_x":"track_episode", "track.external_ids.isrc_x":"track_external_ids_isrc", "track.external_urls.spotify_x":"track_external_ids_spotify", "track.href_x":"track_href", "track.id_x":"track_id", "track.is_local_x":"track_is_local", "track.popularity_x":"track_popularity", "track.preview_url_x":"track_preview_url", "track.track_x":"track_track", "track.track_number_x":"track_track_number", "track.type_x":"track_type", "video_thumbnail.url_x":"video_thumbnail_url", "external_urls.spotify_x":"external_urls_spotify", "followers.href_x":"followers_href", "followers.total_x":"followers_total", "owner.display_name_x":"owner_display_name", "owner.external_urls.spotify_x":"owner_external_urls_spotify", "owner.href_x":"owner_href", "owner.id_x":"owner_id", "owner.type_x":"owner_type", "owner.uri_x":"owner_uri", "tracks.href_x":"tracks_href", "tracks.limit_x":"tracks_limit", "tracks.next_x":"tracks_next", "tracks.offset_x":"tracks_offset", "tracks.previous_x":"tracks_previous", "tracks.total_x":"tracks_total","name_y":"playlist_name", "track.name_y":"track_name", "track.uri_y":"track_uri", "track.album.name_y":"album_name", "track.explicit_y":"isExplicit", "track.album.release_date_y":"release_date", "track.duration_ms_y":"duration_ms", "track.album.uri_y":"album_uri", "added_by.external_urls.spotify_y": "added_by_external_urls_spotify", 'added_by.href_y':"added_by_href", "added_by.id_y":"added_by_id", "added_by.type_y":"added_by_type", "added_by.uri_y":"added_by_uri", "track.album.album_type_y":"track_album_album_type", "track.album.external_urls.spotify_y":"track_album_external_urls_spotify", "track.album.href_y":"track_album_href", "track.album.id_y":"track_album_id", "track.album.release_date_precision_y":"track_album_release_date_precision", "track.album.total_tracks_y":"track_album_total_tracks","track.album.type_y":"track_album_type", "track.disc_number_y":"track_disc_number", "track.episode_y":"track_episode", "track.external_ids.isrc_y":"track_external_ids_isrc", "track.external_urls.spotify_y":"track_external_ids_spotify", "track.href_y":"track_href", "track.id_y":"track_id", "track.is_local_y":"track_is_local", "track.popularity_y":"track_popularity", "track.preview_url_y":"track_preview_url", "track.track_y":"track_track", "track.track_number_y":"track_track_number", "track.type_y":"track_type", "video_thumbnail.url_y":"video_thumbnail_url", "external_urls.spotify_y":"external_urls_spotify", "followers.href_y":"followers_href", "followers.total_y":"followers_total", "owner.display_name_y":"owner_display_name", "owner.external_urls.spotify_y":"owner_external_urls_spotify", "owner.href_y":"owner_href", "owner.id_y":"owner_id", "owner.type_y":"owner_type", "owner.uri_y":"owner_uri", "tracks.href_y":"tracks_href", "tracks.limit_y":"tracks_limit", "tracks.next_y":"tracks_next", "tracks.offset_y":"tracks_offset", "tracks.previous_y":"tracks_previous", "tracks.total_y":"tracks_total", "added_at_x":"added_at", "added_at_y":"added_at", "is_local_x":"is_local", "is_local_y":"is_local"})
        tracks = tracks.drop(columns=['track.album.artists_x', 'track.album.artists_y', 'track.album.available_markets_x', 'track.album.images_x', 'track.artists_x', 'track.available_markets_x', 'track.available_markets_y', 'track.artists_y', 'track.album.images_y','track.album.available_markets_y', 'track.album.images_y', 'images_x', 'tracks.items_x'], errors='ignore')
        load_batch.append(tracks)
        
load_batch = dd.multi.concat(load_batch)
load_batch.head()

ValueError: Metadata mismatch found in `from_delayed`.

Partition type: `pandas.core.frame.DataFrame`
+-------------------------------------+-------+----------+
| Column                              | Found | Expected |
+-------------------------------------+-------+----------+
| 'track.album.external_urls.spotify' | -     | object   |
| 'track.album.total_tracks'          | -     | int64    |
| 'track.episode'                     | -     | bool     |
| 'track.external_ids.isrc'           | -     | object   |
| 'track.external_urls.spotify'       | -     | object   |
| 'track.track'                       | -     | bool     |
+-------------------------------------+-------+----------+

In [79]:
load_batch = []
series = dd.from_pandas(new_batch, npartitions=4)
series = series['playlist_uri'].unique()
if len(tracks) >= 100:
        data2 = sp.playlist_tracks(playlist, offset=100)['items']
        dfs2 = [dask.delayed(pd.json_normalize)(x) for x in data2]
        tracks2 = dd.from_delayed(dfs2)
        data3 = sp.playlist_tracks(playlist, offset=200)['items']
        dfs3 = [dask.delayed(pd.json_normalize)(x) for x in data3]
        tracks3 = dd.from_delayed(dfs3)
        tracks = dd.multi.concat([tracks, tracks2, tracks3])
        secondary_data = sp.playlist_tracks(playlist, offset=100)['items']
        dfs4 = [dask.delayed(pd.json_normalize)(x) for x in secondary_data]
        secondary_data = dd.from_delayed(dfs4, meta=pd.DataFrame(secondary_data[1]))
        secondary_data['uri'] = playlist
        tracks['playlist_uri'] = playlist
        tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
        tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
        tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items', 'track.episode', 'track.external_ids.isrc', 'track.external_urls.spotify', 'track.track'], errors='ignore')

        load_batch.append(tracks)
else:
        secondary_data = sp.playlist(playlist)
        dfs4 = [dask.delayed(pd.json_normalize(x)) for x in secondary_data]
        secondary_data = dd.from_delayed(dfs4, meta=pd.DataFrame(secondary_data[1]))
        secondary_data['uri'] = playlist
        tracks['playlist_uri'] = playlist
        print(secondary_data.columns)
        tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri', how='inner')
        tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
        tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items', 'track.episode', 'track.external_ids.isrc', 'track.external_urls.spotify', 'track.track'], errors='ignore')
        load_batch.append(tracks)
        
load_batch = dd.multi.concat(load_batch)

100%|██████████| 5/5 [00:05<00:00,  1.04s/it]


ValueError: No objects to concatenate